In [1]:
import itertools
import warnings
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
df_blocks = pd.read_csv('../Data/block_data_ready_cleaned_timeseries.csv')

df_blocks.sort_values('block_timestamp', inplace=True)

df_blocks.set_index('block_timestamp', inplace=True)

ts = df_blocks['proposer_total_reward_in_eth'].dropna()


In [3]:
# Selecting the target variable
y = df_blocks['proposer_total_reward_in_eth']

# Splitting the dataset into train and test sets
split = int(len(y) * 0.8)
train, test = y[0:split], y[split:len(y)]

# Fit an ARIMA model
# The order (p,d,q) needs to be determined using model diagnostics or domain knowledge
model = ARIMA(train, order=(5,1,0)) # Example: order=(5,1,0)
model_fit = model.fit()

# Forecast
forecast = model_fit.forecast(steps=len(test))

# Evaluate the model
rmse = sqrt(mean_squared_error(test, forecast))
print(f'ARIMA Model RMSE: {rmse}')

c:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Python310\lib\site-packages\statsmodels\tsa\base\

ARIMA Model RMSE: 2.535908084494488


In [13]:
# Assuming df is your DataFrame and 'proposer_total_reward_in_eth' is your time series column
ts = df_blocks['proposer_total_reward_in_eth'].dropna()  # Ensure no NaN values

# Check for stationarity and determine 'd'
def check_stationarity(timeseries):
    result = adfuller(timeseries)
    if result[1] > 0.05:
        return 1  # Series is not stationary
    else:
        return 0  # Series is stationary

d = check_stationarity(ts)

# Define the p, d and q parameters to take any value between 0 and 2
p = q = range(0, 3)

# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p, [d], q))

# Grid search for the optimal ARIMA parameters
best_aic = float("inf")
best_pdq = None
best_model = None

In [15]:
#warnings.filterwarnings("ignore")  # Suppress warnings

for param in pdq:
    try:
        model = ARIMA(ts, order=param)
        results = model.fit()
        if results.aic < best_aic:
            best_aic = results.aic
            best_pdq = param
            best_model = results
    except:
        continue

print('Best ARIMA{} model - AIC:{}'.format(best_pdq, best_aic))

Best ARIMA(2, 0, 2) model - AIC:2047292.0072070062


In [16]:
# Selecting the target variable
y = df_blocks['proposer_total_reward_in_eth']

# Splitting the dataset into train and test sets
split = int(len(y) * 0.8)
train, test = y[0:split], y[split:len(y)]

# Fit an ARIMA model
# The order (p,d,q) needs to be determined using model diagnostics or domain knowledge
final_arima_model = ARIMA(train, order=(2,0,2)) # Example: order=(5,1,0)
model_fit = final_arima_model.fit()

# Forecast
forecast = model_fit.forecast(steps=len(test))

# Evaluate the model
rmse = sqrt(mean_squared_error(test, forecast))
print(f'ARIMA Model RMSE: {rmse}')

ARIMA Model RMSE: 2.5357580009008074


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Test')
plt.plot(test.index, forecast, label='Forecast', color='red')
plt.legend()
plt.show()